In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD

In [ ]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 3000
EMBEDDING_DIM = 400
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 1000
LATENT_DIM = 25

In [ ]:
dataset = pd.read_csv('../input/how-i-met-your-mother-vietnamese-sub/HowIMetYourMother_Vietnamese_sub.csv')

In [ ]:
dataset.info()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.info()

In [ ]:
dataset['SENTENCE']

In [ ]:
X = dataset['SENTENCE'].values
y = dataset['LAST_WORD'].values
all_sentences = np.concatenate((X,y), axis = 0) 

In [ ]:
X = np.delete(X, y.argmax())
y = np.delete(y, y.argmax())

In [ ]:
print("max length of X: ", X[X.argmax()])
print("max length of X: ", y[X.argmax()])
print("max length of y: ", X[y.argmax()])
print("max length of y: ", y[y.argmax()])

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(all_sentences)
X = tokenizer.texts_to_sequences(X)
y = tokenizer.texts_to_sequences(y)
word2idx = tokenizer.word_index

In [ ]:
num_words = len(word2idx)+1
print(num_words)

In [ ]:
max_length_X = max(len(s) for s in X)
print('Max sequence length X:', max_length_X)

In [ ]:
max_sequence_length = min(max_length_X, MAX_SEQUENCE_LENGTH)
X = pad_sequences(X, maxlen=max_sequence_length, padding='post')
print('Shape of input tensor:', X.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=num_words)
print("Shape of target tensor", y.shape)

In [ ]:
assert(X.shape[0]==y.shape[0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
import tensorflow as tf


model = Sequential()
model.add(Embedding(num_words, max_length_X, input_length=max_length_X))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(num_words, activation='softmax'))
# compiling the networkfrom keras.optimizers import Adam
adam = Adam(lr = 0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
his = model.fit(X,y,epochs = 50,verbose=1,validation_split=0.2)

In [ ]:
def prepare_input(input_):
    input_ = input_.strip().lower()
    input_ = tokenizer.texts_to_sequences([input_])
    input_  = pad_sequences(input_, max_length_X, padding = 'post')
    print(input_)
    return input_

In [ ]:
raw_input = "chúng tôi thấy "
paded_input = prepare_input(raw_input)
preds = model.predict(paded_input).argmax()
pred_word = tokenizer.index_word[preds]
print("Next word for %s: %s" % (raw_input, pred_word))